In [31]:
# doodleverse_utils Written by Dr Daniel Buscombe, Marda Science LLC
from doodleverse_utils.prediction_imports import *
from doodleverse_utils.imports import *
#=======================================================
# This script was written by Sharon Fitzpatrick
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Model
from keras.metrics import BinaryAccuracy, FalseNegatives, FalsePositives, TrueNegatives,TruePositives, Precision, Recall, AUC

USE_GPU = False

if USE_GPU == True:
   ##use the first available GPU
   os.environ['CUDA_VISIBLE_DEVICES'] = '0' #'1'
else:
   ## to use the CPU (not recommended):
   os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

#suppress tensorflow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

weights='C:/1_USGS/CoastSeg/repos/6_sniffer-classifer/Sniffer-Classifier/Dan_model/weights/resunet/ct_ortho_all_water_768.h5'


In [2]:
W=[]
W.append(weights)

M= []; C=[]; T = []
for counter,weights in enumerate(W):
    print(f"{counter}:{weights} ")
    try:
        configfile = weights.replace('_fullmodel.h5','.json').replace('weights', 'config')
        with open(configfile) as f:
            config = json.load(f)
            # print(config)
    except:
        configfile = weights.replace('.h5','.json').replace('weights', 'config')
        with open(configfile) as f:
            config = json.load(f)
            # print(config)

    for k in config.keys():
        expression=k+'=config["'+k+'"]'
        # print(expression)
        # print(k)
        exec(k+'=config["'+k+'"]')
        # This exec dynamically creates all variables out of all the keys in config
    print('.....................................')
    print('Creating and compiling model {}...'.format(counter))
    if MODEL =='resunet':
        model =  custom_resunet((TARGET_SIZE[0], TARGET_SIZE[1], N_DATA_BANDS),
                        FILTERS,
                        nclasses=[NCLASSES+1 if NCLASSES==1 else NCLASSES][0],
                        kernel_size=(KERNEL,KERNEL),
                        strides=STRIDE,
                        dropout=DROPOUT,#0.1,
                        dropout_change_per_layer=DROPOUT_CHANGE_PER_LAYER,#0.0,
                        dropout_type=DROPOUT_TYPE,#"standard",
                        use_dropout_on_upsampling=USE_DROPOUT_ON_UPSAMPLING,#False,
                        )
    try:
        model = tf.keras.models.load_model(weights)
    except:
        # model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [mean_iou, dice_coef])
        model.compile(optimizer = 'adam', loss = dice_coef_loss, metrics = [mean_iou, dice_coef])
        model.load_weights(weights)

        M.append(model)
        C.append(configfile)
        T.append(MODEL)

metadatadict = {}
metadatadict['model_weights'] = W
metadatadict['config_files'] = C
metadatadict['model_types'] = T

0:C:/1_USGS/CoastSeg/repos/6_sniffer-classifer/Sniffer-Classifier/Dan_model/weights/resunet/ct_ortho_all_water_768.h5 
.....................................
Creating and compiling model 0...


In [6]:
def getLayerIndexByName(model, layername):
    for idx, layer in enumerate(model.layers):
        if layer.name == layername:
            return idx

In [ ]:
with open(configfile) as f:
    config = json.load(f)
config

In [16]:
model.layers[-1].activation
last_layer=model.get_layer('conv2d_29')
model.get_layer("add_4").get_config()

{'name': 'add_4', 'trainable': True, 'dtype': 'float32'}

In [17]:
# When the model is initially loaded all of the layers are trainable
model.summary(show_trainable=True)

Model: "model"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                Y          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 Y          
 alization)                                                                                              

In [18]:
# Get the last encoder layer in the base model
# Get the index of this layer so that we can examine the output
idx=getLayerIndexByName(model,"add_4")
idx

56

In [21]:
# Examine the configuration of the add_4 layer
print(model.layers[idx].get_config())
# Examine the output of the add_4 layer
print(model.layers[idx].output)
# Remove all the encoding layers after the add_4 layer
cut_base_model=Model(inputs=model.inputs,outputs=model.layers[idx].output)

{'name': 'add_4', 'trainable': True, 'dtype': 'float32'}
KerasTensor(type_spec=TensorSpec(shape=(None, 48, 48, 96), dtype=tf.float32, name=None), name='add_4/add:0', description="created by layer 'add_4'")


In [24]:
# See all the layers in the cut base model to false, so that the weights will not be modified later
cut_base_model.summary(show_trainable=True)

Model: "model_6"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               Y          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                Y          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 Y          
 alization)                                                                                            

In [25]:
# Keep the base_model for transfer learning
cut_base_model.trainable=False

In [26]:
cut_base_model.compile(optimizer = 'adam', loss = dice_coef_loss)

In [27]:
# Validate that the trainable attribute of the batch_normalization layer is set to false
cut_base_model.get_layer("batch_normalization_12").trainable

False

In [28]:
# The base model requires that inputs have the shape (None,768,768,3)
img_shape = (768, 768)
inputs=keras.Input(img_shape+(3,))
inputs

<KerasTensor: shape=(None, 768, 768, 3) dtype=float32 (created by layer 'input_2')>

In [29]:
# Append the new classifying layers to the base model
x=cut_base_model.output
x= keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(64,activation='relu')(x)
x = keras.layers.Dense(1,activation="sigmoid")(x)

new_model = Model(inputs=cut_base_model.input,outputs=x)

In [30]:
new_model.summary(show_trainable=True)

Model: "model_7"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               N          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                N          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 N          
 alization)                                                                                            

In [32]:
metrics=[TruePositives(name='tp'),FalsePositives(name='fp'), TrueNegatives(name='tn'), FalseNegatives(name='fn'), BinaryAccuracy(name="BinaryAccuracy"), Precision(name="precision"), Recall(name="recall"),AUC(name="AUC")]
new_model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              metrics=metrics)


In [33]:
new_model.summary(show_trainable=True)

Model: "model_7"
_____________________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     Trainable  
 input_1 (InputLayer)           [(None, 768, 768, 3  0           []                               N          
                                )]                                                                           
                                                                                                             
 conv2d (Conv2D)                (None, 768, 768, 6)  78          ['input_1[0][0]']                N          
                                                                                                             
 batch_normalization (BatchNorm  (None, 768, 768, 6)  24         ['conv2d[0][0]']                 N          
 alization)                                                                                            

In [34]:
def save_model_data(model_name:str,model:'keras.engine.functional.Functional'):
    # Create a directory to store all the model data
    model_save_path=os.getcwd()+os.sep+"models"
    if not os.path.exists(model_save_path):
        os.mkdir(model_save_path)
    model_save_path=os.getcwd()+os.sep+"models"+os.sep+model_name
    if os.path.exists(model_save_path):
        raise FileExistsError(f"This folder '{model_name}' already exists: {model_save_path}.\n Please rename your model.")
    else:
        os.mkdir(model_save_path)
    # Save the actual model
    model.save(model_save_path+os.sep+"model")

In [35]:
# Enter the name of your model
model_name="transfer_model_v_2_0"
# This might take a few seconds
save_model_data(model_name,new_model)

INFO:tensorflow:Assets written to: c:\1_USGS\CoastSeg\repos\6_sniffer-classifer\Sniffer-Classifier\models\transfer_model_v_2_0\model\assets
